In [22]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv


load_dotenv('./env/.env')


#llm = OpenAI()
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

#a = lim.predict("Hello, how are you?")
b = chat.predict("Hello, please say Hi")



b

'Hi! How can I assist you today?'